# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234801602849                   -0.50    8.0         
  2   -7.250207418338       -1.81       -1.39    1.0   5.23ms
  3   -7.251022936122       -3.09       -1.97    1.0   5.16ms
  4   -7.251202187525       -3.75       -2.00    3.0   7.15ms
  5   -7.251330877985       -3.89       -2.63    1.0   5.18ms
  6   -7.251332639931       -5.75       -2.76    1.0   5.30ms
  7   -7.251338661500       -5.22       -3.58    1.0   5.34ms
  8   -7.251338785018       -6.91       -3.98    2.0   6.66ms
  9   -7.251338797324       -7.91       -4.57    1.0   5.55ms
 10   -7.251338798274       -9.02       -4.79    2.0   6.56ms
 11   -7.251338798652       -9.42       -5.30    1.0   5.63ms
 12   -7.251338798699      -10.33       -5.72    2.0   6.68ms
 13   -7.251338798704      -11.30       -6.23    2.0   6.82ms
 14   -7.251338798704      -12.40       -6.78    2.0   6.64ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.036230747305766864
[ Info: Arnoldi iteration step 2: normres = 0.7386453982863651
[ Info: Arnoldi iteration step 3: normres = 0.6362371474177683
[ Info: Arnoldi iteration step 4: normres = 0.5968843015304178
[ Info: Arnoldi iteration step 5: normres = 0.10966578788469114
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.07e-02, 3.45e-02, 5.12e-02, 7.96e-02, 3.82e-02)
[ Info: Arnoldi iteration step 6: normres = 0.24874117102717158
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.33e-03, 6.61e-03, 6.26e-02, 2.16e-01, 2.52e-02)
[ Info: Arnoldi iteration step 7: normres = 0.40323571148462
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (7.74e-04, 9.25e-03, 3.90e-01, 2.60e-02, 8.63e-02)
[ Info: Arnoldi iteration step 8: normres = 0.111873477258059
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (4.27e-05,